In [1]:
from openai import OpenAI
import csv
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

In [2]:
client = OpenAI(
    api_key="YOUR_OPENAI_KEY",
)
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def GPT4(model, prompts, student_answer, output=None):
    messages = [{"role": "user", "content": prompts[0]+student_answer}]
    if output:
        messages.append({"role": "assistant", "content": output})
        messages.append({"role": "user", "content": prompts[1]})
    response_json = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    output = response_json.choices[0].message.content
    return output

In [3]:
question = 'Q1'
model = 'gpt-4-1106-preview'
exp_name = 'rationale_score_onequery'
dataset_path = f'sample_answer/{question}.csv'
output_path = f'gpt-4-output/{question}.csv'

In [4]:
prompts = [open(f'prompt/{exp_name}/{question}.txt').read()]
if exp_name == 'rationale_score_twoqueries':
    prompts.append('Assign a score based on your judgment.')
answers = pd.read_csv(dataset_path, header=0)
answers

,Id,EssaySet,EssayText,EssayScore
0,1061.0,1.0,First you would need to know the amount of vin...,2.0
1,1717.0,1.0,How much vinegar do you put into the container...,1.0
2,910.0,1.0,Additional information needed in order to repl...,1.0
3,851.0,1.0,To replicate the experiment I would need the d...,0.0


In [5]:
outputs = []
for i, stu in answers.iterrows():
    output = GPT4(model, prompts, stu.EssayText)
    if exp_name == 'rationale_score_twoqueries':
        outputs.append('\n\n'.join([output, GPT4(model, prompts, stu.EssayText, output)]))
    else:
        outputs.append(output)

In [6]:
answers[model] = outputs
answers

,Id,EssaySet,EssayText,EssayScore,gpt-4-1106-preview
0,1061.0,1.0,First you would need to know the amount of vin...,2.0,The student's response successfully identifies...
1,1717.0,1.0,How much vinegar do you put into the container...,1.0,Judgment:\nThe student response provides one c...
2,910.0,1.0,Additional information needed in order to repl...,1.0,Judgment:\nThe student's response provides two...
3,851.0,1.0,To replicate the experiment I would need the d...,0.0,The student's response is vague and does not p...


In [7]:
answers.to_csv(output_path, index=False)